In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from aif360.datasets import AdultDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    balanced_accuracy_score,
    confusion_matrix
 )

from IPython.display import Markdown, display

sns.set_theme(style="whitegrid")
%matplotlib inline


## Load Dataset

In [ ]:
# privileged and unprivileged groups
privileged_groups = [{'sex': 1, 'age_binary': 1}] # old white males
unprivileged_groups = [{'sex': 0, 'age_binary': 0}]

### Custom preprocessing pipeline
To align with the fairness experiments, we explicitly control how the Adult dataset encodes protected attributes. The block below binarizes age around the dataset median, maps race and sex to binary indicators, and removes the original age column so that downstream models only see the derived protected features.

In [ ]:
# custom processing for the dataset
def custom_preprocessing(df):
    """Binarize age, encode race/sex, and drop raw columns to expose protected attributes explicitly."""
    median_age = df['age'].median()
    df['age_binary'] = (df['age'] > median_age).astype(float)
    df.drop(columns=['age'], inplace=True)
    df['race'] = (df['race'] == 'White').astype(float)
    df['sex'] = (df['sex'] == 'Male').astype(float)
    return df

dataset = AdultDataset(custom_preprocessing=custom_preprocessing,
                              protected_attribute_names=['age_binary', 'sex'],
                              privileged_classes=[np.array([1.0]), np.array([1.0])] ) # old white males

# Get the dataset and split into train and test
np.random.seed(1)
dataset_orig_train, dataset_orig_vt = dataset.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

#### Clean up training data

In [ ]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

In [ ]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [ ]:
print('The 50 first instance weights originally:')
print(dataset.instance_weights[:50])

print('The 50 first instance weights after reweighing:')
dataset_transf_train.instance_weights[:50]

In [ ]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

## Train classifier on original/weighted data

In [ ]:
# --- Extract features and labels ---
def extract_xy(dataset):
    """Return numpy arrays of features and flattened labels for AIF360 datasets."""
    return dataset.features, dataset.labels.ravel()

In [ ]:
X_train, y_train = extract_xy(dataset_orig_train)
X_valid, y_valid = extract_xy(dataset_orig_valid)
X_test,  y_test  = extract_xy(dataset_orig_test)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled  = scaler.transform(X_test)

X_train_transf, y_train_transf = extract_xy(dataset_transf_train)
w_train_transf = dataset_transf_train.instance_weights
X_train_transf_scaled = scaler.fit_transform(X_train_transf)

In [ ]:
# Train logistic regression on the original split
lr_classifier_origin = LogisticRegression(max_iter=1000)
lr_classifier_origin.fit(X_train_scaled, y_train)
y_test_orig_pred = lr_classifier_origin.predict(X_test_scaled)

# Train the same model on reweighed data using instance weights
lr_classifier_transf = LogisticRegression(max_iter=1000)
lr_classifier_transf.fit(X_train_transf_scaled, y_train_transf,
                         sample_weight=w_train_transf)
y_test_transf_pred = lr_classifier_transf.predict(X_test_scaled)

In [ ]:
def summarize_fairness_metrics(metric):
    """Collect commonly used fairness metrics from an AIF360 ClassificationMetric object."""
    return {
        "Statistical Parity Difference": metric.statistical_parity_difference(),
        "Disparate Impact": metric.disparate_impact(),
        "Average Odds Difference": metric.average_odds_difference(),
        "Equal Opportunity Difference": metric.equal_opportunity_difference(),
        "Balanced Accuracy (TPR/TNR avg)": 0.5 * (metric.true_positive_rate() + metric.true_negative_rate())
    }

def plot_model_performance(model_name, y_true, y_pred):
    """Visualize scalar performance metrics and confusion matrix for a classifier."""
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0)
    }
    metrics_df = pd.DataFrame({
        "Metric": list(metrics.keys()),
        "Value": list(metrics.values())
    })
    cm = confusion_matrix(y_true, y_pred)

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    sns.barplot(data=metrics_df, x="Metric", y="Value", hue="Metric", dodge=False, palette="viridis", ax=axes[0])
    axes[0].set_ylim(0, 1)
    axes[0].set_title(f"{model_name} Metrics")
    axes[0].set_ylabel("Score")
    axes[0].tick_params(axis='x', rotation=45)
    legend = axes[0].get_legend()
    if legend is not None:
        legend.remove()

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[1])
    axes[1].set_title(f"{model_name} Confusion Matrix")
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("Actual")

    plt.tight_layout()
    return metrics

def plot_fairness_comparison(metrics_a, metrics_b, labels=("No Reweighing", "Reweighing")):
    """Plot side-by-side fairness metrics for two models to compare mitigation impact."""
    fairness_df = pd.DataFrame({
        "Metric": list(metrics_a.keys()) + list(metrics_b.keys()),
        "Value": list(metrics_a.values()) + list(metrics_b.values()),
        "Model": [labels[0]] * len(metrics_a) + [labels[1]] * len(metrics_b)
    })

    fig, ax = plt.subplots(figsize=(10, 4))
    sns.barplot(data=fairness_df, x="Metric", y="Value", hue="Model", ax=ax)
    ax.set_title("Fairness Metric Comparison")
    ax.tick_params(axis='x', rotation=45)
    ax.set_ylabel("Value")
    ax.axhline(0, color='black', linewidth=0.8)
    plt.tight_layout()

In [ ]:
# Evaluate models
print("MODEL EVALUATION DASHBOARD")

# Visual diagnostics for the original and reweighed models
metrics_orig = plot_model_performance("No Reweighing", y_test, y_test_orig_pred)
metrics_transf = plot_model_performance("Reweighing", y_test, y_test_transf_pred)

# Fairness metrics for both models
dataset_orig_test_pred = dataset_orig_test.copy()
dataset_orig_test_pred.labels = y_test_orig_pred
metric_orig_test = ClassificationMetric(dataset_orig_test, dataset_orig_test_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)
fairness_metrics_orig = summarize_fairness_metrics(metric_orig_test)

dataset_transf_test_pred = dataset_orig_test.copy()
dataset_transf_test_pred.labels = y_test_transf_pred
metric_transf_test = ClassificationMetric(dataset_orig_test, dataset_transf_test_pred,
                                          unprivileged_groups=unprivileged_groups,
                                          privileged_groups=privileged_groups)
fairness_metrics_transf = summarize_fairness_metrics(metric_transf_test)

# Compare fairness metrics side-by-side with a bar plot and styled table
plot_fairness_comparison(fairness_metrics_orig, fairness_metrics_transf)
fairness_table = pd.DataFrame({
    "Metric": list(fairness_metrics_orig.keys()),
    "No Reweighing": list(fairness_metrics_orig.values()),
    "Reweighing": list(fairness_metrics_transf.values())
})
display(fairness_table.style.format({"No Reweighing": "{:.4f}", "Reweighing": "{:.4f}"}).set_caption("Fairness metrics summary"))

### Why statistical parity difference is not exactly zero
Reweighing rebalances the training distribution but does not hard-constrain the downstream classifier. Logistic regression is still optimized for accuracy on the reweighted data, so slight disparities can reappear once the model is evaluated on unseen validation/test splits with their original distributions. Small residual SPD values therefore signal that the mitigation reduced—yet did not entirely eliminate—the gap. Further mitigation (e.g., stronger fairness regularizers or post-processing) would be required to drive SPD closer to zero.

In [ ]:
print("COMPARISON")
print(f"Original Model Accuracy:    {accuracy_score(y_test, y_test_orig_pred):.4f}")
print(f"Transformed Model Accuracy: {accuracy_score(y_test, y_test_transf_pred):.4f}")
print(f"Accuracy Difference:        {accuracy_score(y_test, y_test_transf_pred) - accuracy_score(y_test, y_test_orig_pred):.4f}")
print(f"Original Model SPD:         {metric_orig_test.statistical_parity_difference():.4f}")
print(f"Transformed Model SPD:      {metric_transf_test.statistical_parity_difference():.4f}")
print(f"SPD Improvement:            {abs(metric_transf_test.statistical_parity_difference()) - abs(metric_orig_test.statistical_parity_difference()):.4f}")
print("Note: Statistical Parity Difference closer to 0 indicates better fairness")

### Fairness requirements checklist
- **Protected attributes**: Age (binarized) and Sex are explicitly encoded and tracked through AIF360 datasets, satisfying the requirement to focus on these groups.
- **Classifier + fairness metric**: A baseline logistic regression model is trained and evaluated with statistical parity difference, disparate impact, average odds difference, and equal opportunity difference.
- **Mitigation technique**: Reweighing is applied to the training data, producing a fairer model whose metrics are directly compared to the baseline.
- **Reporting**: Visual dashboards plus the comparison table document the before/after fairness metrics, showing measurable improvement even if SPD is not exactly zero.

✅ *Conclusion:* the fairness analysis portion of the project statement (pre-privacy) is complete. Next steps will involve repeating the same metric/mitigation workflow on the privacy-preserving dataset once it is available.